# Part 16: Analysis of scRNAseq data from Kallionpaa et al., 2019

In [ ]:
source("diabetes_analysis_v07.R")

rank_score_func <- function(df){
df <- df %>% mutate(score = -1*log(p_val_adj+(10^-310))*avg_log2FC*(pct.1/(pct.2+10^-300)))
return(df)
}



# Loading and preprocessing data

This is a reanalysis of data published in [Kallionpaa et al., 2019](https://diabetesjournals.org/diabetes/article-split/68/10/2024/35360/Early-Detection-of-Peripheral-Blood-Cell-Signature). The raw data files were downloaded after approval by the data accession commitee from the EGA archive [EGAD00001005768](https://ega-archive.org/datasets/EGAD00001005768) and mapped and counted using 10x Cellranger software v7.1.0 and the GRCh38 human transcriptome.

In [ ]:
# List paths for reading individual datasets
file_paths <- paste0(list.dirs("../../../Project scRNAseq/Analysis of previously published data/068_Kallionpaa_diabimmune/scRNAseq/raw/cellranger", 
                       recursive = F),"/outs/filtered_feature_bc_matrix")

In [ ]:
# Use the short name for sample identification
file_paths2  <- list.dirs("../../../Project scRNAseq/Analysis of previously published data/068_Kallionpaa_diabimmune/scRNAseq/raw/cellranger", 
                       recursive = F, full.names = F)

In [ ]:
file_paths2

The function process_dataset takes the input files and file names specified in file_paths and file_paths2, it uses the Read10X function to load the datasets, create Seurat objects, calculate the percentage of mitochondrial and ribosomal genes and SCTransformation.
    The individual datasets are saved in a temp_data folder and named with a suffix "_full.rds".

In [ ]:
full_dataset2 <- map(1:length(file_paths2),process_dataset)

In [ ]:
full_dataset <- Merge_Seurat_List(full_dataset2)

In [ ]:
options(future.globals.maxSize = 10000 * 1024^2)

full_dataset <- NormalizeData(full_dataset, verbose = FALSE)
full_dataset <- ScaleData(full_dataset, verbose = FALSE)
full_dataset <- FindVariableFeatures(full_dataset, nfeatures = 1000, verbose = FALSE)
full_dataset <- RunPCA(full_dataset)
full_dataset <- RunUMAP(full_dataset, dims = 1:10)
full_dataset <- FindNeighbors(full_dataset)
full_dataset <- FindClusters(full_dataset, resolution = 0.2)
saveRDS(full_dataset, "../data/Kallionpaa_2019/kallionpaa_full.rds")

DimPlot(full_dataset, group.by = "source")
DimPlot(full_dataset, label = T)

FeaturePlot(full_dataset, features = "nCount_RNA")
FeaturePlot(full_dataset, features = "MKI67")
FeaturePlot(full_dataset, features = "CD4")
FeaturePlot(full_dataset, features = "CD8A")
FeaturePlot(full_dataset, features = "CD3D")
FeaturePlot(full_dataset, features = "CD3D")

### Annotation of cell types

In [ ]:
mid.se <- celldex::MonacoImmuneData()
hpca.se  <- celldex::HumanPrimaryCellAtlasData()

load("../data/ref_wherry_new.RData")

In [ ]:
full_dataset  <- annotate_tcell_data(full_dataset)

In [ ]:
saveRDS(full_dataset, "../data/Kallionpaa_2019/kallionpaa_full.rds")

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 9)
DimPlot(full_dataset, group.by = "Monaco_single", label = T, repel = T)

In [ ]:
full_dataset_filt  <- subset(full_dataset, seurat_clusters %in% c(0:3,5))

In [ ]:
options(future.globals.maxSize = 10000 * 1024^2)
full_dataset_filt <- SCTransform(full_dataset_filt)
full_dataset_filt <- RunPCA(full_dataset_filt)
full_dataset_filt <- RunUMAP(full_dataset_filt, dims = 1:10)
full_dataset_filt <- FindNeighbors(full_dataset_filt)
full_dataset_filt <- FindClusters(full_dataset_filt, resolution = 0.2)
saveRDS(full_dataset_filt, "kalinopaa_full_filt.rds")

DimPlot(full_dataset_filt, group.by = "source")
DimPlot(full_dataset_filt, label = T)

FeaturePlot(full_dataset_filt, features = "nCount_RNA")
FeaturePlot(full_dataset_filt, features = "MKI67")
FeaturePlot(full_dataset_filt, features = "CD4")
FeaturePlot(full_dataset_filt, features = "CD8A")
FeaturePlot(full_dataset_filt, features = "CD3D")
FeaturePlot(full_dataset_filt, features = "CD3D")

In [ ]:
FeaturePlot(full_dataset_filt, features = c("MKI67", "CD8A"))


In [ ]:
full_dataset_filt <- FindClusters(full_dataset_filt, resolution = 0.7)

In [ ]:
DimPlot(full_dataset_filt, label = T)

In [ ]:
DimPlot(full_dataset_filt, label = T, group.by = "Monaco_single", repel = T)

In [ ]:
options(repr.plot.width = 20)
VlnPlot(full_dataset_filt, features = c("percent.mt", "percent.rp", "nCount_RNA", "nFeature_RNA"), ncol = 4)

### Remove dead cells

In [ ]:
cutoff_nFeature_RNA <- 500
cutoff_percent_mt <- 7
cluster_exclude <- c(5,8)

In [ ]:
p1 <- ggplot(data.frame(nCount_RNA = full_dataset_filt$nCount_RNA,
                  nFeature_RNA = full_dataset_filt$nFeature_RNA,
                  percent_mt = full_dataset_filt$percent.mt,
                  seurat_clusters = full_dataset_filt$seurat_clusters,
                  exclude = ifelse(full_dataset_filt$seurat_clusters %in% cluster_exclude, TRUE, FALSE)), 
       aes(x = seurat_clusters, y = percent_mt)) +
  geom_violin(scale = "width", aes(fill = exclude)) + 
  geom_hline(yintercept = cutoff_percent_mt,
               geom = "line", 
               width = 0.5,
               colour = "red") + 
  ggtitle("Percent mt. cutoff") + 
  theme_classic() +
  scale_fill_manual(values = c("white","red")) +
  theme(panel.background = element_blank(), 
        axis.text.x = element_text(angle = 0, hjust = 1)) +
  annotate(geom = "rect", xmin = min(as.numeric(full_dataset_filt$seurat_clusters))-1, 
           xmax = max(as.numeric(full_dataset_filt$seurat_clusters))+1, 
           ymin=cutoff_percent_mt,ymax=1.1*(max(full_dataset_filt$percent.mt)), fill = "red", alpha = 0.1)

p2 <- ggplot(data.frame(nCount_RNA = full_dataset_filt$nCount_RNA,
                  nFeature_RNA = full_dataset_filt$nFeature_RNA,
                  percent_mt = full_dataset_filt$percent.mt,
                  seurat_clusters = full_dataset_filt$seurat_clusters,
                        exclude = ifelse(full_dataset_filt$seurat_clusters %in% cluster_exclude, TRUE, FALSE)), 
       aes(x = seurat_clusters, y = nFeature_RNA)) +
  geom_violin(scale = "width", aes(fill = exclude)) + 
  geom_hline(yintercept = cutoff_nFeature_RNA,
               geom = "line", 
               width = 0.5,
               colour = "red") + 
  ggtitle("nFeature RNA cutoff") + 
  theme_classic() +
  scale_fill_manual(values = c("white","red")) +
  theme(panel.background = element_blank(), 
        axis.text.x = element_text(angle = 0, hjust = 1)) +
  annotate(geom = "rect", xmin = min(as.numeric(full_dataset_filt$seurat_clusters))-1, 
           xmax = max(as.numeric(full_dataset_filt$seurat_clusters))+1, 
           ymin=0, ymax=cutoff_nFeature_RNA, fill = "red", alpha = 0.1)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 5)
p1 + p2

In [ ]:
 full_dataset_filt_stacas <- subset(full_dataset_filt, 
                       ((seurat_clusters %in% cluster_exclude) == F) &
                      percent.mt < cutoff_percent_mt &
                      nFeature_RNA > cutoff_nFeature_RNA)

In [ ]:
merged.list  <- SplitObject(full_dataset_filt_stacas, split.by = "source")

## STACAS Integration over Patient

In [ ]:
plan("sequential")

In [ ]:
# normalize and identify variable features for each dataset independently
merged.list <- lapply(X = merged.list, FUN = function(x) {
    DefaultAssay(x)  <- "RNA"
    x$barcode  <- colnames(x)
    x <- NormalizeData(x)
    x <- FindVariableFeatures(x, selection.method = "vst", nfeatures = 2000)
})

library(STACAS)

full_dataset_filt_stacas <- Run.STACAS(merged.list, dims = 1:12)
full_dataset_filt_stacas <- RunUMAP(full_dataset_filt_stacas, dims = 1:12) 

In [ ]:
# Visualize

DimPlot(full_dataset_filt_stacas, group.by = c("source"))

In [ ]:
saveRDS(full_dataset_filt_stacas, "kalinopaa_filt_stacas.rds")

In [ ]:
DefaultAssay(full_dataset_filt_stacas)  <- "RNA"

In [ ]:
FeaturePlot(full_dataset_filt_stacas, features = "FOXP3", min.cutoff = 0, max.cutoff = 1)

In [ ]:
FeaturePlot(full_dataset_filt_stacas, features = "MKI67", min.cutoff = 0, max.cutoff = 1)

In [ ]:
FeaturePlot(full_dataset_filt_stacas, features = "IL32", min.cutoff = 0, max.cutoff = 1)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

FeaturePlot(full_dataset_filt_stacas, features = c("CD4", "FOXP3", "CD44", "CCL5", "TBX21", "IFNG", "PDCD1", "BCL6"), ncol = 4)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

FeaturePlot(full_dataset_filt_stacas, features = c("SELL", "ZBTB16", "BHLHE40", "FOXP3", "IL2RA", "ZEB2", "ZEB1", "CSF2"), ncol = 4)

In [ ]:
full_dataset_filt_stacas@meta.data  <- full_dataset_filt_stacas@meta.data  %>% mutate(Condition = substr(source,1,4))

In [ ]:
# Visualize

DimPlot(full_dataset_filt_stacas, group.by = c("source"))

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5)
DimPlot(full_dataset_filt_stacas, group.by = c("Condition"))

In [ ]:
DefaultAssay(full_dataset_filt_stacas)  <- "RNA"

In [ ]:
FeaturePlot(full_dataset_filt_stacas, features = "CD8A", min.cutoff = 0, max.cutoff = 1)

In [ ]:
FeaturePlot(full_dataset_filt_stacas, features = "FOXP3", min.cutoff = 0, max.cutoff = 1)

In [ ]:
FeaturePlot(full_dataset_filt_stacas, features = "CD4", min.cutoff = 0, max.cutoff = 1)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

FeaturePlot(full_dataset_filt_stacas, features = c("CD4", "FOXP3", "CD44", "CCL5", "TBX21", "IFNG", "PDCD1", "BCL6"), ncol = 4)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)
DimPlot(full_dataset_filt_stacas, label = T)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 6)
DimPlot(full_dataset_filt_stacas, label = T, group.by = "Monaco_single", repel = T)

In [ ]:
saveRDS(full_dataset_filt_stacas, "kalinopaa_filt_stacas.rds")